In [1]:
import logging
from pymatgen.ext.matproj import MPRester
import numpy as np

In [2]:
key="Q0tUKnAE52sy7hVO"
# Get a structure from the materials project
a = MPRester(key)
testStructure = a.get_structure_by_material_id('mp-1143') 
# struct = a.get_structure_by_material_id('mp-1143', conventional_unit_cell=True)#I'll try supercell for now to visualize easier.

In [3]:
testStructure

Structure Summary
Lattice
    abc : 5.177955250310783 5.177955250310783 5.17795522336617
 angles : 55.28961178917173 55.28961178917173 55.28961598024499
 volume : 87.42003699644115
      A : 4.586845 -2.402514 0.0
      B : 4.586845 2.402514 0.0
      C : 3.328448 0.0 3.966441
PeriodicSite: Al (1.8491, 0.0000, 0.5867) [0.1479, 0.1479, 0.1479]
PeriodicSite: Al (4.4020, 0.0000, 1.3966) [0.3521, 0.3521, 0.3521]
PeriodicSite: Al (8.1002, 0.0000, 2.5699) [0.6479, 0.6479, 0.6479]
PeriodicSite: Al (10.6530, 0.0000, 3.3798) [0.8521, 0.8521, 0.8521]
PeriodicSite: O (7.7124, 0.9315, 0.9916) [0.5561, 0.9439, 0.2500]
PeriodicSite: O (5.6629, -0.7355, 0.2227) [0.7500, 0.4439, 0.0561]
PeriodicSite: O (6.8392, 0.7355, 3.7437) [0.2500, 0.5561, 0.9439]
PeriodicSite: O (7.3271, -1.6670, 2.2059) [0.9439, 0.2500, 0.5561]
PeriodicSite: O (5.1750, 1.6670, 1.7605) [0.0561, 0.7500, 0.4439]
PeriodicSite: O (4.7898, -0.9315, 2.9748) [0.4439, 0.0561, 0.7500]

## ChemEnv analysis 

Potential problem:
    
    The Number of connections found by this code is less than what they are found by the print_links method of ChemEnv. 
    
    We might not (?) directly compare it to the Robocrystal results as they take symmetry into account. Robocrys finds the same number of connections as print_links, but for ALL the symmetry group.

Potential salvation:

    This code (basically from Robocrys) skips atoms which have already been counted as a conncetion. Since Multiple connectoins can exist, this might explain the fewer number found.

In [4]:
from pymatgen.analysis.bond_valence import BVAnalyzer
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import SimplestChemenvStrategy
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments
from pymatgen.analysis.chemenv.utils.defs_utils import AdditionalConditions
from pymatgen.analysis.chemenv.connectivity.connectivity_finder import ConnectivityFinder

In [6]:
#get the valences
bv = BVAnalyzer()
valences = bv.get_valences(testStructure)

In [7]:
# Setup the local geometry finder
lgf = LocalGeometryFinder()
logging.basicConfig(  # filename='chemenv_structure_environments.log',
    format='%(levelname)s:%(module)s:%(funcName)s:%(message)s',
    level=logging.WARNING)
lgf.setup_structure(structure=testStructure)


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



In [8]:
# Get the StructureEnvironments
#please test the additional conditions again. There was a change in the code, I think. It's now a list, which I find a bit confusing.#
# just avoid that there is bug
se = lgf.compute_structure_environments(only_cations=True, valences=valences, additional_conditions=[AdditionalConditions.ONLY_ANION_CATION_BONDS])

In [9]:
#Select a strategy - we can keep the simple one here for now
#I would keep these parameters for now but we can check at a later stage.
strategy = SimplestChemenvStrategy(distance_cutoff=1.4, angle_cutoff=0.3)

In [10]:
#retrieve the light structure environments with less information
lse = LightStructureEnvironments.from_structure_environments(strategy=strategy, structure_environments=se)


cf= ConnectivityFinder()
sc=cf.get_structure_connectivity(light_structure_environments=lse)
sc.print_links()

Links in graph :
0  is connected with : 
  - 1 by 3 ligands (0 0 0)
  - 1 by 1 ligands (0 0 -1)
  - 1 by 1 ligands (0 -1 0)
  - 1 by 1 ligands (-1 0 0)
  - 2 by 1 ligands (-1 0 0)
  - 2 by 1 ligands (0 -1 -1)
  - 2 by 1 ligands (-1 -1 0)
  - 2 by 1 ligands (0 -1 0)
  - 2 by 1 ligands (-1 0 -1)
  - 2 by 1 ligands (0 0 -1)
  - 3 by 2 ligands (-1 -1 0)
  - 3 by 2 ligands (0 -1 -1)
  - 3 by 2 ligands (-1 0 -1)
1  is connected with : 
  - 0 by 3 ligands (0 0 0)
  - 0 by 1 ligands (0 0 1)
  - 0 by 1 ligands (0 1 0)
  - 0 by 1 ligands (1 0 0)
  - 2 by 2 ligands (0 0 -1)
  - 2 by 2 ligands (-1 0 0)
  - 2 by 2 ligands (0 -1 0)
  - 3 by 1 ligands (0 0 -1)
  - 3 by 1 ligands (-1 -1 0)
  - 3 by 1 ligands (0 -1 -1)
  - 3 by 1 ligands (-1 0 -1)
  - 3 by 1 ligands (0 -1 0)
  - 3 by 1 ligands (-1 0 0)
2  is connected with : 
  - 0 by 1 ligands (1 0 0)
  - 0 by 1 ligands (0 1 1)
  - 0 by 1 ligands (1 1 0)
  - 0 by 1 ligands (0 1 0)
  - 0 by 1 ligands (1 0 1)
  - 0 by 1 ligands (0 0 1)
  - 1 by 2 ligand

Tne goal is to use the package above (Pymatgen, particularly ChemEnv) to produce features similar to the condensed version of Robocrystalographer from hackingmaterials.

In [11]:
### Code below is mostly from Robocrys

In [18]:
from pymatgen.analysis.local_env import BrunnerNN_real
from pymatgen.util.coord import get_angle

We want a core (pymatgen) function that takes the site and imagne, and returns the distance and/or position. (NNN)

In [21]:
def get_coords(a_site_index, a_site_image, crysSturcture ,printStuff = False):
    if printStuff==True:
        print(a_site_index)
    bnr = BrunnerNN_real()
    return np.asarray(
                bnr.get_bonded_structure(crysSturcture).structure.lattice.get_cartesian_coords(
                    bnr.get_bonded_structure(crysSturcture).structure.frac_coords[a_site_index]
                    + a_site_image
                )
            )

In [74]:
# %%time
def crysFeaturizer(lightStructureEnv, report_atoms = False):
    struct = lightStructureEnv.structure
    struct_data = {}
    
    for atom_index, atom in enumerate(struct):
        if lightStructureEnv.neighbors_sets[atom_index] == None:
            continue  #so we just go over cations, since lightStructureEnv has been only defined for cations
        atom_data = {}
        atom_data['species']=atom.species
        atom_data['abc']=(atom.a, atom.b, atom.c) # we can add xyz instead.

        NN_list = lightStructureEnv.neighbors_sets[atom_index][0].neighb_sites_and_indices
    #     NN_list = bnr.get_nn(struct,atom_index)   # almost identical to lightStructureEnv.neighbors_sets[atom_index][0].neighb_sites_and_indices

    #     NN_list=[nb['site'].distance_and_image(struct[atom_index]) for nb in NN_list] # use this if we don't want all
    #     the neighbors data, only the distance and image

        atom_data['NN_list']=NN_list
        
        bnr = BrunnerNN_real()
    #     bnr.get_nn(struct,atom_index)   # identical to lightStructureEnv.neighbors_sets[atom_index][0].neighb_sites_and_indices
        nn_sites = bnr.get_bonded_structure(struct).get_connected_sites(atom_index) #data on NNs for an atom    
        next_nn_sites = [
    site for nn_site in nn_sites for site in bnr.get_bonded_structure(struct).get_connected_sites(
        nn_site.index, jimage=nn_site.jimage)
            ]  #tells us the data on NNNs for the atom.

        nn_sites_set = set((site.index, site.jimage) for site in nn_sites) #records the index and jimage of NNs    
        seen_nnn_sites = set()  #so we'll skip atoms we've already seen (multiple neighbors)


        next_nn_summary = []

        for nnn_site in next_nn_sites:

            if (
                nnn_site.index == atom_index
                and nnn_site.jimage == (0, 0, 0) # skip the nnn site if it is the original atom of interest
                or (nnn_site.index, nnn_site.jimage) in seen_nnn_sites
            ):
                continue

            seen_nnn_sites.add((nnn_site.index, nnn_site.jimage)) #so we'll skip this atom the next time
#             print('Location 31')

            sites = set(
                (site.index, site.jimage)
                for site in bnr.get_bonded_structure(struct).get_connected_sites(
                    nnn_site.index, jimage=nnn_site.jimage
                ))  #again buidling a set because of repetition.

            shared_sites = nn_sites_set.intersection(sites)

            n_shared_atoms = len(shared_sites)

            if n_shared_atoms == 1:
                connectivity = "corner"
            elif n_shared_atoms == 2:
                connectivity = "edge"
            elif n_shared_atoms >= 3:
                connectivity = "face"

            site_coords = get_coords(atom_index, (0, 0, 0), struct)
            nnn_site_coords = get_coords(nnn_site.index, nnn_site.jimage, struct)
            nn_site_coords = [
                get_coords(nn_site_index, nn_site_image, struct)#,printStuff=True)
                for nn_site_index, nn_site_image in shared_sites  # only for the NNs between the 2 cations
            ]

            # can't just use Structure.get_angles to calculate angles as it
            # doesn't take into account the site image
            angles = [
                get_angle(site_coords - x, nnn_site_coords - x) for x in nn_site_coords
            ]

            distance = np.linalg.norm(site_coords - nnn_site_coords)
            
#             geometry = self.get_site_geometry(nnn_site.index)  #we use our own geometry tools

            summary = {
                "element": str(nnn_site.site.specie),
                "connectivity": connectivity,
#                 "geometry": geometry,
                "angles": angles,
                "distance": distance,
            }

#             if inc_inequivalent_atom_index:
#                 summary["inequiv_index"] = self.equivalent_sites[nnn_site.index]#some geometry based code

            next_nn_summary.append(summary)


        atom_data['NNN_summary']=next_nn_summary
        struct_data[atom_index]=atom_data
        if report_atoms==True:
            print(atom_index, atom)
    return struct_data

In [75]:
%%time
# The distance from atom index i to the NN index j is accessible through:
# struct_data[i]['NN_list'][j]['site'].distance_from_point(struct[i].coords)

# The NN distance of the NN index j can also be accessed as shown below. 
# Not sure if this is always equivalant to the above.
# struct_data[i]['NN_list'][j]['site'].nn_distance

testFeats = crysFeaturizer(lightStructureEnv=lse, report_atoms=True)
testFeats

0 [1.84911622 0.         0.58665249] Al
1 [4.40195278 0.         1.39656801] Al
2 [8.10018522 0.         2.56987299] Al
3 [10.65302178  0.          3.37978851] Al
CPU times: user 13.2 s, sys: 7.93 ms, total: 13.2 s
Wall time: 13.2 s


{0: {'species': Comp: Al1,
  'abc': (0.147904, 0.147904, 0.147904),
  'NN_list': [{'site': PeriodicSite: O (2.7403, 0.7355, 2.2059) [-0.0561, 0.2500, 0.5561],
    'index': 7},
   {'site': PeriodicSite: O (1.4613, -0.9315, -0.9916) [0.4439, 0.0561, -0.2500],
    'index': 9},
   {'site': PeriodicSite: O (0.5882, -0.7355, 1.7605) [0.0561, -0.2500, 0.4439],
    'index': 8},
   {'site': PeriodicSite: O (3.1255, -1.4710, 0.9916) [0.5561, -0.0561, 0.2500],
    'index': 4},
   {'site': PeriodicSite: O (1.0761, 1.6670, 0.2227) [-0.2500, 0.4439, 0.0561],
    'index': 5},
   {'site': PeriodicSite: O (3.5108, 0.7355, -0.2227) [0.2500, 0.5561, -0.0561],
    'index': 6}],
  'NNN_summary': [{'element': 'Al',
    'connectivity': 'corner',
    'angles': [120.36244667800388],
    'distance': 3.2504195105645763},
   {'element': 'Al',
    'connectivity': 'corner',
    'angles': [132.22718116699446],
    'distance': 3.531979198871541},
   {'element': 'Al',
    'connectivity': 'edge',
    'angles': [93.6327

In [27]:
playData = np.load('coordPlayData.npy',allow_pickle=True)

In [28]:
playData = playData[:5] #just to make testing faster

In [90]:
%%time
res= []
for datum in pplayData:
    res.append(crysFeaturizer(datum))#, report_atoms=True)
    
res = np.array(res)

# takes quite a while on the work laptop    

0 [2.757316 0.       5.581613] Cu
1 [0.       4.782881 5.581613] Cu
2 [2.757316 4.782881 0.      ] Cu
3 [0. 0. 0.] Cu
4 [4.02925484 0.12795163 4.14048518] N
5 [1.27193884 4.65492937 7.02274082] N
6 [1.48537716 4.91083263 1.44112782] N
7 [4.24269316 9.43781037 9.72209818] N
8 [1.48537716 9.43781037 7.02274082] N
9 [4.24269316 4.91083263 4.14048518] N
10 [4.02925484 4.65492937 9.72209818] N
11 [1.27193884 0.12795163 1.44112782] N
12 [4.12226462 1.98676094 6.90468971] N
13 [1.36494862 2.79612006 4.25853629] N
14 [1.39236738 6.76964194 9.84014929] N
15 [4.14968338 7.57900106 1.32307671] N
16 [1.39236738 7.57900106 4.25853629] N
17 [4.14968338 6.76964194 6.90468971] N
18 [4.12226462 2.79612006 1.32307671] N
19 [1.36494862 1.98676094 9.84014929] N
CPU times: user 53.1 s, sys: 35.8 ms, total: 53.1 s
Wall time: 53.1 s


{0: {'species': Comp: Cu1,
  'abc': (0.5, 0.0, 0.5),
  'NN_list': [{'site': PeriodicSite: O (1.4830, -0.6385, 4.3346) [0.2689, -0.0667, 0.3883],
    'index': 24},
   {'site': PeriodicSite: O (4.0317, 0.6385, 6.8286) [0.7311, 0.0667, 0.6117],
    'index': 20}],
  'NNN_summary': [{'element': 'N',
    'connectivity': 'corner',
    'angles': [114.70312591799939],
    'distance': 2.7496973929521062},
   {'element': 'N',
    'connectivity': 'corner',
    'angles': [114.70312591799939],
    'distance': 2.7496973929521062}]},
 1: {'species': Comp: Cu1,
  'abc': (0.0, 0.5, 0.5),
  'NN_list': [{'site': PeriodicSite: O (1.2743, 4.1444, 4.3346) [0.2311, 0.4333, 0.3883],
    'index': 21},
   {'site': PeriodicSite: O (-1.2743, 5.4214, 6.8286) [-0.2311, 0.5667, 0.6117],
    'index': 25}],
  'NNN_summary': [{'element': 'N',
    'connectivity': 'corner',
    'angles': [114.70312591799939],
    'distance': 2.7496973929521062},
   {'element': 'N',
    'connectivity': 'corner',
    'angles': [114.70312591